In [4]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

2023-09-26 14:41:44.524415: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-26 14:41:44.598670: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 14:41:45.111403: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-26 14:41:45.113939: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-26 14:41:46.626796: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [59]:
interpreter = tf.lite.Interpreter(model_path='my_cnn_model.tflite')
interpreter.allocate_tensors()

In [60]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_index = input_details[0]['index'] 

In [61]:
input_data = "new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train/Grape___Black_rot/00cff577-afd4-4e36-ac9c-a52aa6ae5949___FAM_B.Rot 0508_flipLR.JPG"

In [64]:
import cv2 
image = cv2.imread(input_data)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 

model_image_size = (256, 256)
resized_image = cv2.resize(image, model_image_size, interpolation = cv2.INTER_CUBIC)
resized_image = resized_image.astype(np.float32)
resized_image /= 255
test_image = np.expand_dims(resized_image, axis=0)


In [65]:
# Set the input tensor
interpreter.set_tensor(input_index, test_image)

# Run inference
interpreter.invoke()

# Get the output tensor
output_details = interpreter.get_output_details()
output_index = output_details[0]['index']
output_data = interpreter.get_tensor(output_index)

In [66]:
input_details

[{'name': 'serving_default_conv2d_10_input:0',
  'index': 0,
  'shape': array([  1, 256, 256,   3], dtype=int32),
  'shape_signature': array([ -1, 256, 256,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [67]:
prediction = output_data.argmax()
prediction

12

In [34]:
import tensorflow as tf


def run_tflite_model(tflite_file, test_image):

    interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_index = input_details[0]['index'] 

    interpreter.set_tensor(input_index, test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    prediction = output.argmax()

    return prediction

In [40]:
import cv2

In [44]:
converted_model = 'plant_disease/AllPlantDiseaseCNN.tflite'
input_data = "new-plant-diseases-dataset/new plant diseases dataset(augmented)/New Plant Diseases Dataset(Augmented)/train/Apple___Cedar_apple_rust/025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655_180deg.JPG"

img = cv2.imread(input_data)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 

resized = cv2.resize(image, (100, 100)).astype('float32')
resized_image = cv2.resize(image, (100, 100), interpolation = cv2.INTER_CUBIC)

test_image = np.expand_dims(test_image, axis=0)
prediction = run_tflite_model(converted_model, test_image)

print(prediction)

[{'name': 'serving_default_conv2d_input:0', 'index': 0, 'shape': array([  1, 100, 100,   3], dtype=int32), 'shape_signature': array([ -1, 100, 100,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


ValueError: Cannot set tensor: Dimension mismatch. Got 5 but expected 4 for input 0.